In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [2]:
df = pd.read_csv('movies.csv')

In [3]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
df.shape

(4803, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
df[:5]['genres']

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
Name: genres, dtype: object

In [7]:
df[:5]['keywords']

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2           spy based on novel secret agent sequel mi6
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
Name: keywords, dtype: object

In [8]:
df[:5]['cast']

0    Sam Worthington Zoe Saldana Sigourney Weaver S...
1    Johnny Depp Orlando Bloom Keira Knightley Stel...
2    Daniel Craig Christoph Waltz L\u00e9a Seydoux ...
3    Christian Bale Michael Caine Gary Oldman Anne ...
4    Taylor Kitsch Lynn Collins Samantha Morton Wil...
Name: cast, dtype: object

### Feature Selection

In [9]:
features = ['overview', 'genres', 'keywords', 'cast', 'director']

In [10]:
for ft in features:
    df[ft] = df[ft].fillna('')
df.isna().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   0
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [11]:
df['genres'] = df['genres'].str.replace(' ', '')
df['keywords'] = df['keywords'].str.replace(' ', '')

In [26]:
combinedFeatures = df['overview']+" "+df['genres'] + " "+df['keywords']+ " "+df['cast']+" "+ df['director']
combinedFeatures

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Length: 4803, dtype: object

In [27]:
tfidf = TfidfVectorizer()
featureVector = tfidf.fit_transform(combinedFeatures)
print(featureVector)

  (0, 5244)	0.14940905236193727
  (0, 16574)	0.0952507060740186
  (0, 26505)	0.16947714649902126
  (0, 20502)	0.1389237093290303
  (0, 18044)	0.19377502143736774
  (0, 29692)	0.13106521321493228
  (0, 33791)	0.17124880829186198
  (0, 28477)	0.17970731123056705
  (0, 27007)	0.18999109444334383
  (0, 34810)	0.1703491340338733
  (0, 34412)	0.20629539470190525
  (0, 27050)	0.12491957859443885
  (0, 7960)	0.2489169756717554
  (0, 601)	0.21341698184683824
  (0, 6306)	0.2006089171886816
  (0, 1431)	0.1520676280978471
  (0, 1714)	0.06137831690052964
  (0, 24786)	0.19377502143736774
  (0, 1750)	0.03684565980415756
  (0, 22714)	0.18512131203986648
  (0, 12341)	0.15714138208854453
  (0, 3751)	0.11703603103429254
  (0, 31517)	0.1670114738829642
  (0, 3447)	0.11164550773990363
  (0, 5053)	0.071672749919461
  :	:
  (4802, 31089)	0.06966336992675756
  (4802, 22266)	0.06549682195147818
  (4802, 14048)	0.08368671403970109
  (4802, 33852)	0.07926362714844908
  (4802, 11049)	0.07863044469464037
  (4802, 

In [28]:
similarity = cosine_similarity(featureVector)

In [29]:
similarity

array([[1.        , 0.0195694 , 0.02502622, ..., 0.02170541, 0.03294001,
        0.00790688],
       [0.0195694 , 1.        , 0.02416674, ..., 0.0295629 , 0.01960864,
        0.01116243],
       [0.02502622, 0.02416674, 1.        , ..., 0.0280694 , 0.05093   ,
        0.01101844],
       ...,
       [0.02170541, 0.0295629 , 0.0280694 , ..., 1.        , 0.03551131,
        0.02298067],
       [0.03294001, 0.01960864, 0.05093   , ..., 0.03551131, 1.        ,
        0.03154876],
       [0.00790688, 0.01116243, 0.01101844, ..., 0.02298067, 0.03154876,
        1.        ]])

In [30]:
similarity.shape

(4803, 4803)

In [31]:
movieList = df['title'].tolist()
movieList

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [ ]:
movie_name = input("Enter your faviourate movie")

In [45]:
closeMatch = difflib.get_close_matches(movie_name,movieList)[0]
closeMatch

'Counting'

In [46]:
index_of_movie = df[df.title==closeMatch]['index'].values[0]
index_of_movie

4755

In [47]:
similarityScore = list(enumerate(similarity[index_of_movie]))
similarityScore

[(0, 0.018441182740286326),
 (1, 0.00352746873513323),
 (2, 0.021082434060771534),
 (3, 0.013762003306610034),
 (4, 0.02528707532157371),
 (5, 0.028172968551487865),
 (6, 0.02109466641416313),
 (7, 0.016902699003677257),
 (8, 0.012725657225392933),
 (9, 0.01325731464269318),
 (10, 0.026350455474161004),
 (11, 0.0035673479864956658),
 (12, 0.0013078217968677897),
 (13, 0.009949040959136776),
 (14, 0.011593623154786684),
 (15, 0.01926286596558761),
 (16, 0.020239038271098485),
 (17, 0.014877575752417743),
 (18, 0.020391216564366908),
 (19, 0.014596011530796142),
 (20, 0.02420375408370718),
 (21, 0.010237771536564022),
 (22, 0.0022195879428598755),
 (23, 0.017254575399586088),
 (24, 0.034628953576355745),
 (25, 0.01454496154724141),
 (26, 0.019521101458528818),
 (27, 0.018816660095682933),
 (28, 0.0010940613490328058),
 (29, 0.008907548364888385),
 (30, 0.011324459464507466),
 (31, 0.013721052417740657),
 (32, 0.03510749823857921),
 (33, 0.0024445050266994513),
 (34, 0.024573933731070998)

In [48]:
similarityList = sorted(similarityScore, key=lambda x: x[1], reverse=True)[:11]

In [49]:
for item in similarityList:
    i = item[0]
    if i==index_of_movie :
        print("Reccomended movies for : ",movie_name)
    else:
        print(df[df.index==i]['title'].values[0])

Reccomended movies for :  burning
The Case of the Grinning Cat
Beauty Shop
I Want Your Money
Peace, Propaganda & the Promised Land
Stake Land
Taxi to the Dark Side
Pixels
The Algerian
Barbershop 2:  Back in Business
2016: Obama's America


In [50]:
def fun(movie_name):
    closeMatch = difflib.get_close_matches(movie_name,movieList)[0]
    index_of_movie = df[df.title==closeMatch]['index'].values[0]
    similarityScore = list(enumerate(similarity[index_of_movie]))
    similarityList = sorted(similarityScore, key=lambda x: x[1], reverse=True)[:11]

    i = 1
    for item in similarityList:
        indx = item[0]
        if indx==index_of_movie :
            print("Reccomended movies for : ",movie_name)
        else:
            print(i, df[df.index==indx]['title'].values[0])
            i+=1

In [52]:
movie_name = input("Enter movie name : ")
fun(movie_name)

Enter movie name :  harry potter


Reccomended movies for :  harry potter
1 The Players Club
2 America Is Still the Place
3 The Amityville Horror
4 Bizarre
5 Shattered Glass
6 Neighbors 2: Sorority Rising
7 The Postman
8 Exotica
9 Inside Deep Throat
10 Austin Powers: International Man of Mystery
